In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import scipy.sparse as sparse

from lightfm import LightFM
import lightgbm as lgb
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
import sklearn
from sklearn.model_selection import train_test_split

import scipy.sparse as sparse

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [2]:
train = pd.read_csv('data/train.csv.zip', low_memory=False)
test = pd.read_csv('data/test.csv.zip', low_memory=False)
sample_submission = pd.read_csv('data/sample_submission.csv.zip', low_memory=False)

In [3]:
train_data, test_data = train_test_split(train,random_state=32, shuffle=True)


In [4]:
ratings_coo = sparse.coo_matrix((train_data['rating'].astype(int),
                                 (train_data['userid'],
                                  train_data['itemid'])))

In [5]:
model = LightFM(learning_rate=0.07, 
                loss='logistic',
                no_components=40, 
                learning_schedule='adagrad',
               random_state = 13)
model = model.fit(ratings_coo, epochs=23, 
                  num_threads=8)

preds = model.predict(test_data.userid.values,
                      test_data.itemid.values)
sklearn.metrics.roc_auc_score(test_data.rating, preds)

0.746993098661152

In [6]:
preds = model.predict(test.userid.values,
                      test.itemid.values)

In [7]:
preds.min(), preds.max()

(-21.338375091552734, 27.7176570892334)

In [8]:
normalized_preds = (preds - preds.min())/(preds - preds.min()).max()

In [9]:
normalized_preds.min(), normalized_preds.max()


(0.0, 1.0)

In [10]:
sample_submission['rating'] = normalized_preds
sample_submission.to_csv('submission_4.csv', index=False)
sample_submission.head(3)

,Id,rating
0,0,0.472014
1,1,0.464880
2,2,0.444784


In [11]:
train_data_GB = pd.DataFrame({
    'userid': train['userid'],
    'itemid': train['itemid'],
    'rating': train['rating']
})

test_data_GB = pd.DataFrame({
    'userid': test['userid'],
    'itemid': test['itemid'],
})

In [12]:
train_data_GB_train, test_data_GB_test = train_test_split(train_data_GB,random_state=13, shuffle=True)

In [13]:
y_train = train_data_GB_train['rating']
y_test = test_data_GB_test['rating']
X_train = train_data_GB_train.drop('rating', axis=1)
X_test = test_data_GB_test.drop('rating', axis=1)

In [14]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'l2', 'l1'},
    'num_leaves': 8,
    'learning_rate': 0.4,
    'feature_fraction': 0.1,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)
print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', sklearn.metrics.roc_auc_score(y_test, y_pred))

y____ =  gbm.predict(test_data_GB, num_iteration=gbm.best_iteration)

Starting training...
[1]	valid_0's l1: 0.256154	valid_0's l2: 0.127638
Training until validation scores don't improve for 10 rounds
[2]	valid_0's l1: 0.256101	valid_0's l2: 0.127574
[3]	valid_0's l1: 0.256047	valid_0's l2: 0.127535
[4]	valid_0's l1: 0.25601	valid_0's l2: 0.127517
[5]	valid_0's l1: 0.255981	valid_0's l2: 0.127506
[6]	valid_0's l1: 0.255963	valid_0's l2: 0.127479
[7]	valid_0's l1: 0.255932	valid_0's l2: 0.127459
[8]	valid_0's l1: 0.255921	valid_0's l2: 0.127457
[9]	valid_0's l1: 0.255893	valid_0's l2: 0.127427
[10]	valid_0's l1: 0.255874	valid_0's l2: 0.127417
[11]	valid_0's l1: 0.255663	valid_0's l2: 0.127398
[12]	valid_0's l1: 0.255517	valid_0's l2: 0.127372
[13]	valid_0's l1: 0.255423	valid_0's l2: 0.12735
[14]	valid_0's l1: 0.25536	valid_0's l2: 0.127341
[15]	valid_0's l1: 0.255313	valid_0's l2: 0.127325
[16]	valid_0's l1: 0.25541	valid_0's l2: 0.127306
[17]	valid_0's l1: 0.255473	valid_0's l2: 0.127303
[18]	valid_0's l1: 0.255511	valid_0's l2: 0.127296
[19]	valid_0'

In [15]:
c = (normalized_preds  + y____) / 2